# Preliminaries

In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import sys
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

sys.path.append("../PaperNon_Gen_LinCFA/LinCFA")
from LinCFA import LinCFA

sys.path.append("../PaperNon_Gen_LinCFA/NonLinCFA")
from NonLinCFA import NonLinCFA

sys.path.append("../PaperNon_Gen_LinCFA/GenLinCFA")
from GenLinCFA import GenLinCFA

sys.path.append("../PaperNon_Gen_LinCFA/droughts/")
from aux import prepare_target,prepare_features,compare_methods



In [37]:
def compute_CI(list,n):
    print(f'{np.mean(list)} +- {1.96*np.std(list)/np.sqrt(n)}')

# Data

In [38]:
### Load and standardize target
target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path='../PaperNon_Gen_LinCFA/droughts/Emiliani1.csv')


target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.379890    0.50  2001     1 -0.382765
1    2001-01-13  0.482679    0.58  2001     2  0.319215
2    2001-01-21  0.516259    0.59  2001     3  0.548542
3    2001-01-29  0.434421    0.50  2001     5 -0.010351
4    2001-02-06  0.494805    0.54  2001     6  0.402030
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.427085    0.43  2009    48 -0.060454
407  2009-12-05  0.547380    0.57  2009    49  0.761079
408  2009-12-13  0.531070    0.58  2009    50  0.649694
409  2009-12-21  0.295704    0.00  2009    52 -0.957702
410  2009-12-29  0.027861    0.00  2009    53 -2.786888

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))


In [39]:
### Load and standardize features
variables_list = ['cyclostationary_mean_tg', 
                 'cyclostationary_mean_tg_1w',
                 'cyclostationary_mean_tg_4w', 
                 'cyclostationary_mean_tg_8w',
                 'cyclostationary_mean_tg_12w', 
                 'cyclostationary_mean_tg_16w',
                 'cyclostationary_mean_tg_24w',
                 'cyclostationary_mean_rr', 
                 'cyclostationary_mean_rr_1w',
                 'cyclostationary_mean_rr_4w', 
                 'cyclostationary_mean_rr_8w',
                 'cyclostationary_mean_rr_12w', 
                 'cyclostationary_mean_rr_16w',
                 'cyclostationary_mean_rr_24w'
                 ]

df_train = pd.DataFrame()
df_val = pd.DataFrame()
df_test = pd.DataFrame()
df_trainVal = pd.DataFrame()

for variable in variables_list:
    df_train_unfolded_std, df_val_unfolded_std, df_test_unfolded_std,df_trainVal_unfolded_std = prepare_features('../PaperNon_Gen_LinCFA/droughts/Emiliani1_aggreg.csv',variable,False,max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')
    df_train_unfolded_std = df_train_unfolded_std.add_prefix(variable)
    df_val_unfolded_std = df_val_unfolded_std.add_prefix(variable)
    df_test_unfolded_std = df_test_unfolded_std.add_prefix(variable)
    df_trainVal_unfolded_std = df_trainVal_unfolded_std.add_prefix(variable)
    df_train = pd.concat((df_train,df_train_unfolded_std),axis=1)
    df_val = pd.concat((df_val,df_val_unfolded_std),axis=1)
    df_test = pd.concat((df_test,df_test_unfolded_std),axis=1)
    df_trainVal = pd.concat((df_trainVal,df_trainVal_unfolded_std),axis=1)
    
df_trainVal

,cyclostationary_mean_tgmean_12.149860342381333_43.74986055078544,cyclostationary_mean_tgmean_12.149860342381333_43.8498605504681,cyclostationary_mean_tgmean_12.149860342381333_43.94986055015075,cyclostationary_mean_tgmean_12.149860342381333_44.04986054983341,cyclostationary_mean_tgmean_12.149860342381333_44.14986054951606,cyclostationary_mean_tgmean_12.149860342381333_44.24986054919872,cyclostationary_mean_tgmean_12.149860342381333_44.34986054888138,cyclostationary_mean_tgmean_12.149860342381333_44.44986054856403,cyclostationary_mean_tgmean_12.149860342381333_44.54986054824669,cyclostationary_mean_tgmean_12.149860342381333_44.64986054792934,...,cyclostationary_mean_rr_24wmean_11.349860345581744_44.849860547294654,cyclostationary_mean_rr_24wmean_11.449860345181692_44.14986054951606,cyclostationary_mean_rr_24wmean_11.449860345181692_44.24986054919872,cyclostationary_mean_rr_24wmean_11.449860345181692_44.34986054888138,cyclostationary_mean_rr_24wmean_11.449860345181692_44.44986054856403,cyclostationary_mean_rr_24wmean_11.449860345181692_44.54986054824669,cyclostationary_mean_rr_24wmean_11.449860345181692_44.64986054792934,cyclostationary_mean_rr_24wmean_11.449860345181692_44.749860547612,cyclostationary_mean_rr_24wmean_11.449860345181692_44.849860547294654,cyclostationary_mean_rr_24wmean_10.749860347982052_44.24986054919872
0,0.719982,0.737633,0.791074,0.848973,0.983768,1.172439,1.201792,1.127842,0.986823,0.794499,...,2.331970,6.168345,6.215139,5.406085,4.240808,4.540304,2.472063,2.105030,2.296531,3.363161
1,1.845931,2.162888,2.278888,2.313132,2.330406,2.363919,2.119857,2.008556,1.797622,1.626867,...,1.670322,3.527914,3.841691,3.108603,2.947959,2.824932,1.593425,1.462249,1.728906,3.004583
2,-0.312107,-0.404970,-0.361520,-0.116017,0.037568,0.199148,0.215971,0.233859,0.106281,-0.117314,...,1.118337,1.915294,2.358315,2.981883,2.858264,2.382361,1.067353,1.006560,1.055914,1.950734
3,1.666516,1.661740,1.601745,1.595421,1.579804,1.588420,1.391649,1.185715,1.141724,0.968893,...,0.534584,2.429799,2.789340,2.202089,1.991279,1.546552,0.466434,0.489244,0.506180,2.157730
4,-0.031846,-0.003213,-0.013352,0.122002,0.295776,0.473944,0.532393,0.421560,0.238369,-0.025356,...,1.056396,2.718182,3.000691,3.082463,3.061446,2.457115,1.281718,1.138486,1.142762,2.350811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,0.578850,0.609135,0.689113,0.967141,1.273710,1.237462,1.007356,1.032989,1.194737,1.011196,...,0.690343,1.041006,0.388026,0.123181,0.347596,0.465708,-0.240666,0.449174,0.650788,1.164679
635,1.243632,1.043466,0.783567,0.922940,1.124030,1.209410,1.205085,1.367888,1.900526,1.924692,...,0.666434,0.955184,0.294397,0.090659,0.410255,0.581733,-0.169828,0.462829,0.652467,1.096017
636,0.267909,0.598843,0.778573,1.011130,1.340522,1.491446,1.432183,1.526572,1.965525,1.990127,...,0.894777,1.047647,0.425352,0.262704,0.623916,0.826442,0.059008,0.692457,0.885935,1.242053
637,0.957833,1.053095,1.249241,1.296098,1.319729,1.135229,0.976383,1.250418,1.632408,1.738173,...,0.878993,1.036936,0.456154,0.304824,0.500595,0.733759,0.026283,0.689393,0.807281,1.119456


In [40]:
### Together
df_trainVal_withTar = pd.concat((df_trainVal,target_df_trainVal['mean_std']), axis=1)
df_test_withTar = pd.concat((df_test,target_df_test.reset_index()['mean_std']), axis=1)

df_trainVal_withTar

,cyclostationary_mean_tgmean_12.149860342381333_43.74986055078544,cyclostationary_mean_tgmean_12.149860342381333_43.8498605504681,cyclostationary_mean_tgmean_12.149860342381333_43.94986055015075,cyclostationary_mean_tgmean_12.149860342381333_44.04986054983341,cyclostationary_mean_tgmean_12.149860342381333_44.14986054951606,cyclostationary_mean_tgmean_12.149860342381333_44.24986054919872,cyclostationary_mean_tgmean_12.149860342381333_44.34986054888138,cyclostationary_mean_tgmean_12.149860342381333_44.44986054856403,cyclostationary_mean_tgmean_12.149860342381333_44.54986054824669,cyclostationary_mean_tgmean_12.149860342381333_44.64986054792934,...,cyclostationary_mean_rr_24wmean_11.449860345181692_44.14986054951606,cyclostationary_mean_rr_24wmean_11.449860345181692_44.24986054919872,cyclostationary_mean_rr_24wmean_11.449860345181692_44.34986054888138,cyclostationary_mean_rr_24wmean_11.449860345181692_44.44986054856403,cyclostationary_mean_rr_24wmean_11.449860345181692_44.54986054824669,cyclostationary_mean_rr_24wmean_11.449860345181692_44.64986054792934,cyclostationary_mean_rr_24wmean_11.449860345181692_44.749860547612,cyclostationary_mean_rr_24wmean_11.449860345181692_44.849860547294654,cyclostationary_mean_rr_24wmean_10.749860347982052_44.24986054919872,mean_std
0,0.719982,0.737633,0.791074,0.848973,0.983768,1.172439,1.201792,1.127842,0.986823,0.794499,...,6.168345,6.215139,5.406085,4.240808,4.540304,2.472063,2.105030,2.296531,3.363161,-0.382765
1,1.845931,2.162888,2.278888,2.313132,2.330406,2.363919,2.119857,2.008556,1.797622,1.626867,...,3.527914,3.841691,3.108603,2.947959,2.824932,1.593425,1.462249,1.728906,3.004583,0.319215
2,-0.312107,-0.404970,-0.361520,-0.116017,0.037568,0.199148,0.215971,0.233859,0.106281,-0.117314,...,1.915294,2.358315,2.981883,2.858264,2.382361,1.067353,1.006560,1.055914,1.950734,0.548542
3,1.666516,1.661740,1.601745,1.595421,1.579804,1.588420,1.391649,1.185715,1.141724,0.968893,...,2.429799,2.789340,2.202089,1.991279,1.546552,0.466434,0.489244,0.506180,2.157730,-0.010351
4,-0.031846,-0.003213,-0.013352,0.122002,0.295776,0.473944,0.532393,0.421560,0.238369,-0.025356,...,2.718182,3.000691,3.082463,3.061446,2.457115,1.281718,1.138486,1.142762,2.350811,0.402030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,0.578850,0.609135,0.689113,0.967141,1.273710,1.237462,1.007356,1.032989,1.194737,1.011196,...,1.041006,0.388026,0.123181,0.347596,0.465708,-0.240666,0.449174,0.650788,1.164679,-0.320516
635,1.243632,1.043466,0.783567,0.922940,1.124030,1.209410,1.205085,1.367888,1.900526,1.924692,...,0.955184,0.294397,0.090659,0.410255,0.581733,-0.169828,0.462829,0.652467,1.096017,0.137056
636,0.267909,0.598843,0.778573,1.011130,1.340522,1.491446,1.432183,1.526572,1.965525,1.990127,...,1.047647,0.425352,0.262704,0.623916,0.826442,0.059008,0.692457,0.885935,1.242053,0.305368
637,0.957833,1.053095,1.249241,1.296098,1.319729,1.135229,0.976383,1.250418,1.632408,1.738173,...,1.036936,0.456154,0.304824,0.500595,0.733759,0.026283,0.689393,0.807281,1.119456,-0.586199


# PCA

In [21]:
# explain 95% of variance
from sklearn.decomposition import PCA
from sklearn.utils import resample

r2 = []
for curr_seed in [0,1,2,3,4]: 
    best_r2 = 0
    best_num = 0
    curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
    curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
    pca = PCA(n_components=0.95)
    trainVal_pca = pd.DataFrame(pca.fit_transform(curr_df_trainVal))
    test_pca = pd.DataFrame(pca.transform(df_test))
    actual_r2 = compare_methods(trainVal_pca, test_pca, curr_df_trainVal_withTar, df_test_withTar, list(trainVal_pca.columns))
    r2.append([curr_seed,actual_r2])
    print(test_pca.shape)
    if actual_r2> best_r2:
        best_r2=actual_r2
        best_num = test_pca.shape
print(best_r2)
print(best_num)

Full aggregate regression train score: 0.43276712775854054, test score: 0.15129590752908562
Aggregate regression train score with FS: 0.43276712775854054, test score: 0.15129590752908562
(228, 29)
Full aggregate regression train score: 0.42937863563841616, test score: 0.17943013844961864
Aggregate regression train score with FS: 0.42937863563841616, test score: 0.17943013844961864
(228, 30)
Full aggregate regression train score: 0.5009936178522946, test score: 0.2542855378683515
Aggregate regression train score with FS: 0.5009936178522946, test score: 0.2542855378683515
(228, 30)
Full aggregate regression train score: 0.48931916345113, test score: 0.13966673239943683
Aggregate regression train score with FS: 0.48931916345113, test score: 0.13966673239943683
(228, 29)
Full aggregate regression train score: 0.40429086841596007, test score: 0.23358767872683373
Aggregate regression train score with FS: 0.40429086841596007, test score: 0.23358767872683373
(228, 29)
0.23358767872683373
(228,

In [22]:
n_pca95 = [29,30,30,29,29]
r2_pca95 = [0.15129590752908562,0.17943013844961864,0.2542855378683515,0.13966673239943683,0.23358767872683373]
compute_CI(n_pca95,5)
compute_CI(r2_pca95,5)

29.4 +- 0.42941448508405017
0.19165319899466526 +- 0.03951627810103481


# Isomap

In [26]:
from sklearn.manifold import Isomap
## r2 = []
for curr_seed in [0,1,2,3,4]: 
    best_r2 = 0
    best_num = 0
    for i in range(50):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = Isomap(n_components=i+1,n_neighbors=10)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        actual_r2 = compare_methods(trainVal_reduced, test_reduced, curr_df_trainVal_withTar, df_test_withTar, list(trainVal_reduced.columns))
        r2.append([curr_seed,actual_r2])
        print(test_reduced.shape)
        if actual_r2> best_r2:
            best_r2=actual_r2
            best_num = test_reduced.shape
    print(best_r2)
    print(best_num)
        

Full aggregate regression train score: 0.18214450589466769, test score: 0.14815560978670061
Aggregate regression train score with FS: 0.18214450589466769, test score: 0.14815560978670061
(228, 1)
Full aggregate regression train score: 0.23825666144373603, test score: 0.2413699932346618
Aggregate regression train score with FS: 0.23825666144373603, test score: 0.2413699932346618
(228, 2)
Full aggregate regression train score: 0.25736458010214125, test score: 0.2892832216319383
Aggregate regression train score with FS: 0.25736458010214125, test score: 0.2892832216319383
(228, 3)
Full aggregate regression train score: 0.26844476959348373, test score: 0.3218331758559365
Aggregate regression train score with FS: 0.26844476959348373, test score: 0.3218331758559365
(228, 4)
Full aggregate regression train score: 0.29008928778005194, test score: 0.29226745423108025
Aggregate regression train score with FS: 0.29008928778005194, test score: 0.29226745423108025
(228, 5)
Full aggregate regression 

Full aggregate regression train score: 0.4968888423550146, test score: 0.12810069486494524
Aggregate regression train score with FS: 0.4968888423550146, test score: 0.12810069486494524
(228, 43)
Full aggregate regression train score: 0.49946020194352336, test score: 0.15067999209717786
Aggregate regression train score with FS: 0.49946020194352336, test score: 0.15067999209717786
(228, 44)
Full aggregate regression train score: 0.5014741542332857, test score: 0.15122837243205323
Aggregate regression train score with FS: 0.5014741542332857, test score: 0.15122837243205323
(228, 45)
Full aggregate regression train score: 0.5019679407450341, test score: 0.14630349424648248
Aggregate regression train score with FS: 0.5019679407450341, test score: 0.14630349424648248
(228, 46)
Full aggregate regression train score: 0.5053487514819592, test score: 0.13471701689834092
Aggregate regression train score with FS: 0.5053487514819592, test score: 0.13471701689834092
(228, 47)
Full aggregate regressi

Full aggregate regression train score: 0.4074932444885778, test score: 0.23504686565360533
Aggregate regression train score with FS: 0.4074932444885778, test score: 0.23504686565360533
(228, 35)
Full aggregate regression train score: 0.444226534425039, test score: 0.22390601210887573
Aggregate regression train score with FS: 0.444226534425039, test score: 0.22390601210887573
(228, 36)
Full aggregate regression train score: 0.4460739713696591, test score: 0.22758122310013273
Aggregate regression train score with FS: 0.4460739713696591, test score: 0.22758122310013273
(228, 37)
Full aggregate regression train score: 0.446522692993893, test score: 0.23188153342136875
Aggregate regression train score with FS: 0.446522692993893, test score: 0.23188153342136875
(228, 38)
Full aggregate regression train score: 0.45303139001607506, test score: 0.22584879015092385
Aggregate regression train score with FS: 0.45303139001607506, test score: 0.22584879015092385
(228, 39)
Full aggregate regression t

Full aggregate regression train score: 0.44957702467625704, test score: 0.2591945702244851
Aggregate regression train score with FS: 0.44957702467625704, test score: 0.2591945702244851
(228, 27)
Full aggregate regression train score: 0.4496097237852835, test score: 0.25814217964404895
Aggregate regression train score with FS: 0.4496097237852835, test score: 0.25814217964404895
(228, 28)
Full aggregate regression train score: 0.4496552345860467, test score: 0.2595275527327985
Aggregate regression train score with FS: 0.4496552345860467, test score: 0.2595275527327985
(228, 29)
Full aggregate regression train score: 0.4510678816269005, test score: 0.25945380659532447
Aggregate regression train score with FS: 0.4510678816269005, test score: 0.25945380659532447
(228, 30)
Full aggregate regression train score: 0.46371326498741505, test score: 0.2822877975832203
Aggregate regression train score with FS: 0.46371326498741505, test score: 0.2822877975832203
(228, 31)
Full aggregate regression t

Full aggregate regression train score: 0.42015671304336744, test score: 0.2323392571414955
Aggregate regression train score with FS: 0.42015671304336744, test score: 0.2323392571414955
(228, 19)
Full aggregate regression train score: 0.42016834600659014, test score: 0.23168758997120165
Aggregate regression train score with FS: 0.42016834600659014, test score: 0.23168758997120165
(228, 20)
Full aggregate regression train score: 0.42074600305527676, test score: 0.2352801267663125
Aggregate regression train score with FS: 0.42074600305527676, test score: 0.2352801267663125
(228, 21)
Full aggregate regression train score: 0.42100917504072777, test score: 0.22806081881550366
Aggregate regression train score with FS: 0.42100917504072777, test score: 0.22806081881550366
(228, 22)
Full aggregate regression train score: 0.42106524086108854, test score: 0.2279188943430729
Aggregate regression train score with FS: 0.42106524086108854, test score: 0.2279188943430729
(228, 23)
Full aggregate regres

Full aggregate regression train score: 0.31268212626844805, test score: 0.2686293650923487
Aggregate regression train score with FS: 0.31268212626844805, test score: 0.2686293650923487
(228, 11)
Full aggregate regression train score: 0.3310006086550519, test score: 0.30083830396014766
Aggregate regression train score with FS: 0.3310006086550519, test score: 0.30083830396014766
(228, 12)
Full aggregate regression train score: 0.3310054973700729, test score: 0.3007767245861943
Aggregate regression train score with FS: 0.3310054973700729, test score: 0.3007767245861943
(228, 13)
Full aggregate regression train score: 0.3323066190327806, test score: 0.2918734508386468
Aggregate regression train score with FS: 0.3323066190327806, test score: 0.2918734508386468
(228, 14)
Full aggregate regression train score: 0.3323253690364366, test score: 0.29443369256070717
Aggregate regression train score with FS: 0.3323253690364366, test score: 0.29443369256070717
(228, 15)
Full aggregate regression tra

In [27]:
best_isomap_n = [50,50,50,50,12]

best_isomap_r2 = [0.3218331758559365,0.3220852621150284,0.35109134562834865,0.31237645902922284,0.30083830396014766]

compute_CI(best_isomap_r2,5)
compute_CI(best_isomap_n,5)


0.3216449093177368 +- 0.014594518320913947
42.4 +- 13.323387437134746


# Supervised PCA

In [28]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn import utils
from sklearn.preprocessing import KernelCenterer, scale
from sklearn.metrics.pairwise import pairwise_kernels
from scipy import linalg
from scipy.sparse.linalg import eigsh as ssl_eigsh

class spca(BaseEstimator, TransformerMixin):
    
    def __init__(self, num_components, kernel="linear", eigen_solver='auto', 
                 max_iterations=None, gamma=0, degree=3, coef0=1, alpha=1.0, 
                 tolerance=0, fit_inverse_transform=False):
        
        self._num_components = num_components
        self._gamma = gamma
        self._tolerance = tolerance
        self._fit_inverse_transform = fit_inverse_transform
        self._max_iterations = max_iterations
        self._degree = degree
        self._kernel = kernel
        self._eigen_solver = eigen_solver
        self._coef0 = coef0
        self._centerer = KernelCenterer()
        self._alpha = alpha
        self._alphas = []
        self._lambdas = []
        
        
    def _get_kernel(self, X, Y=None):
        # Returns a kernel matrix K such that K_{i, j} is the kernel between the ith and jth vectors 
        # of the given matrix X, if Y is None. 
        
        # If Y is not None, then K_{i, j} is the kernel between the ith array from X and the jth array from Y.
        
        # valid kernels are 'linear, rbf, poly, sigmoid, precomputed'
        
        args = {"gamma": self._gamma, "degree": self._degree, "coef0": self._coef0}
        
        return pairwise_kernels(X, Y, metric=self._kernel, n_jobs=-1, filter_params=True, **args)
    
    
    
    def _fit(self, X, Y):
        
        # calculate kernel matrix of the labels Y and centre it and call it K (=H.L.H)
        K = self._centerer.fit_transform(self._get_kernel(Y))
        
        # deciding on the number of components to use
        if self._num_components is not None:
            num_components = min(K.shape[0], self._num_components)
        else:
            num_components = self.K.shape[0]
        
        # Scale X
        # scaled_X = scale(X)
        
        # calculate the eigen values and eigen vectors for X^T.K.X
        Q = (X.T).dot(K).dot(X)
        
        # If n_components is much less than the number of training samples, 
        # arpack may be more efficient than the dense eigensolver.
        if (self._eigen_solver=='auto'):
            if (Q.shape[0]/num_components) > 20:
                eigen_solver = 'arpack'
            else:
                eigen_solver = 'dense'
        else:
            eigen_solver = self._eigen_solver
        
        if eigen_solver == 'dense':
            # Return the eigenvalues (in ascending order) and eigenvectors of a Hermitian or symmetric matrix.
            self._lambdas, self._alphas = linalg.eigh(Q, eigvals=(Q.shape[0] - num_components, Q.shape[0] - 1))
            # argument eigvals = Indexes of the smallest and largest (in ascending order) eigenvalues
        
        elif eigen_solver == 'arpack':
            # deprecated :: self._lambdas, self._alphas = utils.arpack.eigsh(A=Q, num_components, which="LA", tol=self._tolerance)
            self._lambdas, self._alphas = ssl_eigsh(A=Q, k=num_components, which="LA", tol=self._tolerance)
            
        indices = self._lambdas.argsort()[::-1]
        
        self._lambdas = self._lambdas[indices]
        self._lambdas = self._lambdas[self._lambdas > 0]  # selecting values only for non zero eigen values
        
        self._alphas = self._alphas[:, indices]
        #return self._alphas
        self._alphas = self._alphas[:, self._lambdas > 0]  # selecting values only for non zero eigen values
        
        self.X_fit = X

        
    def _transform(self):
        return self.X_fit.dot(self._alphas)
        
        
    def transform(self, X):
        return X.dot(self._alphas)
        
        
    def fit(self, X, Y):
        self._fit(X,Y)
        return
        
        
    def fit_and_transform(self, X, Y):
        self.fit(X, Y)
        return self._transform()
    

In [29]:
## r2 = []
for kernel in ['linear', 'poly', 'sigmoid']:
    for curr_seed in [0,1,2,3,4]: 
        best_r2 = 0
        best_num = 0
        for i in range(50):
            curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
            curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
            dimRedMethod = spca(num_components=i+1, kernel='linear', degree=3, gamma=None, coef0=1)
            trainVal_reduced = pd.DataFrame(dimRedMethod.fit_and_transform(curr_df_trainVal.values,curr_df_trainVal_withTar.mean_std.values.reshape(-1, 1)))
            test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
            actual_r2 = compare_methods(trainVal_reduced, test_reduced, curr_df_trainVal_withTar, df_test_withTar, list(trainVal_reduced.columns))
            r2.append([curr_seed,actual_r2])
            print(test_reduced.shape)
            if actual_r2> best_r2:
                best_r2=actual_r2
                best_num = test_reduced.shape
        print(best_r2)
        print(best_num)
        print(f'#########################{kernel}, {curr_seed}#############################')
    

Full aggregate regression train score: 0.29284730437617223, test score: 0.32996110795946043
Aggregate regression train score with FS: 0.29284730437617223, test score: 0.32996110795946043
(228, 1)
Full aggregate regression train score: 0.3194166357212287, test score: 0.3504867616297538
Aggregate regression train score with FS: 0.3194166357212287, test score: 0.3504867616297538
(228, 2)
Full aggregate regression train score: 0.33083251520514056, test score: 0.24990430162441435
Aggregate regression train score with FS: 0.33083251520514056, test score: 0.24990430162441435
(228, 3)
Full aggregate regression train score: 0.33347946066923884, test score: 0.3546942743279401
Aggregate regression train score with FS: 0.33347946066923884, test score: 0.3546942743279401
(228, 4)
Full aggregate regression train score: 0.3517805908400513, test score: 0.3562544166694822
Aggregate regression train score with FS: 0.3517805908400513, test score: 0.3562544166694822
(228, 5)
Full aggregate regression trai

Full aggregate regression train score: 0.4665902148683131, test score: 0.10711966318984678
Aggregate regression train score with FS: 0.4665902148683131, test score: 0.10711966318984678
(228, 44)
Full aggregate regression train score: 0.45117391167014675, test score: 0.10765114452011759
Aggregate regression train score with FS: 0.45117391167014675, test score: 0.10765114452011759
(228, 45)
Full aggregate regression train score: 0.47675164983283247, test score: 0.08172193704377706
Aggregate regression train score with FS: 0.47675164983283247, test score: 0.08172193704377706
(228, 46)
Full aggregate regression train score: 0.46436965958219556, test score: -0.17415128456958762
Aggregate regression train score with FS: 0.46436965958219556, test score: -0.17415128456958762
(228, 47)
Full aggregate regression train score: 0.4831349863299623, test score: -0.03711436031884885
Aggregate regression train score with FS: 0.4831349863299623, test score: -0.03711436031884885
(228, 48)
Full aggregate 

Full aggregate regression train score: 0.458789192335185, test score: 0.009334073750076777
Aggregate regression train score with FS: 0.458789192335185, test score: 0.009334073750076777
(228, 36)
Full aggregate regression train score: 0.4520095269551908, test score: -0.2813678254876788
Aggregate regression train score with FS: 0.4520095269551908, test score: -0.2813678254876788
(228, 37)
Full aggregate regression train score: 0.45061261239428485, test score: 0.24372952635657819
Aggregate regression train score with FS: 0.45061261239428485, test score: 0.24372952635657819
(228, 38)
Full aggregate regression train score: 0.484427637235493, test score: 0.0806107084098292
Aggregate regression train score with FS: 0.484427637235493, test score: 0.0806107084098292
(228, 39)
Full aggregate regression train score: 0.4713467882653025, test score: 0.05813342997216531
Aggregate regression train score with FS: 0.4713467882653025, test score: 0.05813342997216531
(228, 40)
Full aggregate regression t

Full aggregate regression train score: 0.4451931188339098, test score: 0.23890048347201587
Aggregate regression train score with FS: 0.4451931188339098, test score: 0.23890048347201587
(228, 28)
Full aggregate regression train score: 0.4664532906604767, test score: 0.1312826323314794
Aggregate regression train score with FS: 0.4664532906604767, test score: 0.1312826323314794
(228, 29)
Full aggregate regression train score: 0.4701568865420175, test score: 0.2853150748560138
Aggregate regression train score with FS: 0.4701568865420175, test score: 0.2853150748560138
(228, 30)
Full aggregate regression train score: 0.47951403329676423, test score: 0.18602477510861193
Aggregate regression train score with FS: 0.47951403329676423, test score: 0.18602477510861193
(228, 31)
Full aggregate regression train score: 0.4782602769082974, test score: 0.1396489264109705
Aggregate regression train score with FS: 0.4782602769082974, test score: 0.1396489264109705
(228, 32)
Full aggregate regression tra

Full aggregate regression train score: 0.39785271436208414, test score: 0.21755919617050912
Aggregate regression train score with FS: 0.39785271436208414, test score: 0.21755919617050912
(228, 20)
Full aggregate regression train score: 0.42806987222485404, test score: 0.10212741035522
Aggregate regression train score with FS: 0.42806987222485404, test score: 0.10212741035522
(228, 21)
Full aggregate regression train score: 0.4026291331359443, test score: 0.14624918751813554
Aggregate regression train score with FS: 0.4026291331359443, test score: 0.14624918751813554
(228, 22)
Full aggregate regression train score: 0.42779178186996836, test score: 0.21544485914688538
Aggregate regression train score with FS: 0.42779178186996836, test score: 0.21544485914688538
(228, 23)
Full aggregate regression train score: 0.44592992585894187, test score: 0.1301236317329949
Aggregate regression train score with FS: 0.44592992585894187, test score: 0.1301236317329949
(228, 24)
Full aggregate regression

...

Full aggregate regression train score: 0.48106206773620774, test score: 0.05955288930269653
Aggregate regression train score with FS: 0.48106206773620774, test score: 0.05955288930269653
(228, 49)
Full aggregate regression train score: 0.506113278961901, test score: -0.12660834839977242
Aggregate regression train score with FS: 0.506113278961901, test score: -0.12660834839977242
(228, 50)
0.40753966149722676
(228, 5)
#########################sigmoid, 0#############################
Full aggregate regression train score: 0.29488284807853626, test score: 0.3032988473087862
Aggregate regression train score with FS: 0.29488284807853626, test score: 0.3032988473087862
(228, 1)
Full aggregate regression train score: 0.2948873320426798, test score: 0.30380794345110507
Aggregate regression train score with FS: 0.2948873320426798, test score: 0.30380794345110507
(228, 2)
Full aggregate regression train score: 0.3145332901598672, test score: 0.29981330082057045
Aggregate regression train score wi

Full aggregate regression train score: 0.469727406495696, test score: 0.05534022560445617
Aggregate regression train score with FS: 0.469727406495696, test score: 0.05534022560445617
(228, 41)
Full aggregate regression train score: 0.5007716668465407, test score: 0.026014377076095108
Aggregate regression train score with FS: 0.5007716668465407, test score: 0.026014377076095108
(228, 42)
Full aggregate regression train score: 0.44885854072195286, test score: 0.03182957075355797
Aggregate regression train score with FS: 0.44885854072195286, test score: 0.03182957075355797
(228, 43)
Full aggregate regression train score: 0.46365392452217125, test score: -0.06783065840345914
Aggregate regression train score with FS: 0.46365392452217125, test score: -0.06783065840345914
(228, 44)
Full aggregate regression train score: 0.4986842157067922, test score: -0.036663766785898666
Aggregate regression train score with FS: 0.4986842157067922, test score: -0.036663766785898666
(228, 45)
Full aggregate 

Full aggregate regression train score: 0.4672597038001951, test score: 0.08762822402818182
Aggregate regression train score with FS: 0.4672597038001951, test score: 0.08762822402818182
(228, 33)
Full aggregate regression train score: 0.4998903943840678, test score: 0.29564362910529074
Aggregate regression train score with FS: 0.4998903943840678, test score: 0.29564362910529074
(228, 34)
Full aggregate regression train score: 0.496479611481517, test score: 0.16513947016605024
Aggregate regression train score with FS: 0.496479611481517, test score: 0.16513947016605024
(228, 35)
Full aggregate regression train score: 0.47935696525052773, test score: 0.06672900774532298
Aggregate regression train score with FS: 0.47935696525052773, test score: 0.06672900774532298
(228, 36)
Full aggregate regression train score: 0.5035539371430287, test score: 0.16379659087858744
Aggregate regression train score with FS: 0.5035539371430287, test score: 0.16379659087858744
(228, 37)
Full aggregate regression

Full aggregate regression train score: 0.42005342889429675, test score: 0.07653537993547488
Aggregate regression train score with FS: 0.42005342889429675, test score: 0.07653537993547488
(228, 25)
Full aggregate regression train score: 0.46055458671865945, test score: 0.10831629734852932
Aggregate regression train score with FS: 0.46055458671865945, test score: 0.10831629734852932
(228, 26)
Full aggregate regression train score: 0.46884521402664725, test score: -0.006113937622043375
Aggregate regression train score with FS: 0.46884521402664725, test score: -0.006113937622043375
(228, 27)
Full aggregate regression train score: 0.45598266707882096, test score: 0.12501568980348876
Aggregate regression train score with FS: 0.45598266707882096, test score: 0.12501568980348876
(228, 28)
Full aggregate regression train score: 0.47263241100951203, test score: -0.061210372296523596
Aggregate regression train score with FS: 0.47263241100951203, test score: -0.061210372296523596
(228, 29)
Full ag

Full aggregate regression train score: 0.33462393366616083, test score: 0.2596592433822178
Aggregate regression train score with FS: 0.33462393366616083, test score: 0.2596592433822178
(228, 17)
Full aggregate regression train score: 0.3563685491089874, test score: 0.3470602395656769
Aggregate regression train score with FS: 0.3563685491089874, test score: 0.3470602395656769
(228, 18)
Full aggregate regression train score: 0.3675718896562421, test score: 0.32840262863604197
Aggregate regression train score with FS: 0.3675718896562421, test score: 0.32840262863604197
(228, 19)
Full aggregate regression train score: 0.3526832661826391, test score: 0.32342568136415495
Aggregate regression train score with FS: 0.3526832661826391, test score: 0.32342568136415495
(228, 20)
Full aggregate regression train score: 0.374897304218605, test score: 0.1983946587898302
Aggregate regression train score with FS: 0.374897304218605, test score: 0.1983946587898302
(228, 21)
Full aggregate regression train

In [30]:
linear_r2 = [0.42208233658902483,0.35984883608375096,0.3925300734023428,0.3515224087720409,0.37441931296855646]
poly_r2 = [0.4093980139121096,0.36349576247651094,0.34563826556520305,0.3787237027021739,0.34996937587000176]
sigmoid_r2 = [0.40753966149722676,0.40738148999918466,0.343909602546953,0.36058012493670677,0.3980555995908387]

linear_n = [14,8,4,4,8]
poly_n = [4,16,2,4,12]
sigmoid_n = [5,12,3,6,11]

compute_CI(linear_r2,5)
compute_CI(poly_r2,5)
compute_CI(sigmoid_r2,5)

compute_CI(linear_n,5)
compute_CI(poly_n,5)
compute_CI(sigmoid_n,5)

0.38008059356314317 +- 0.022091834600161456
0.36944502410519986 +- 0.020231877672618408
0.38349329571418195 +- 0.023034182802616383
7.6 +- 3.21344376020493
7.6 +- 4.755979478509132
7.4 +- 3.0666328114073256


# Kernel PCA

In [34]:
from sklearn.decomposition import KernelPCA
## r2 = []

for curr_kernel in ['linear', 'poly', 'sigmoid']:
    for curr_seed in [0,1,2,3,4]: 
        best_r2 = 0
        best_num = 0
        for i in range(50):
            curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
            curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
            dimRedMethod = KernelPCA(n_components=i+1, kernel=curr_kernel)
            trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal))
            test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
            actual_r2 = compare_methods(trainVal_reduced, test_reduced, curr_df_trainVal_withTar, df_test_withTar, list(trainVal_reduced.columns))
            r2.append([curr_seed,actual_r2])
            print(test_reduced.shape)
            if actual_r2> best_r2:
                best_r2=actual_r2
                best_num = test_reduced.shape
        print(best_r2)
        print(best_num)
        print(f'#########################{curr_kernel}, {curr_seed}#############################')


Full aggregate regression train score: 0.20393664673354062, test score: 0.22075836574179564
Aggregate regression train score with FS: 0.20393664673354062, test score: 0.22075836574179564
(228, 1)
Full aggregate regression train score: 0.2544317154648672, test score: 0.2839297390852117
Aggregate regression train score with FS: 0.2544317154648672, test score: 0.2839297390852117
(228, 2)
Full aggregate regression train score: 0.32785662848978125, test score: 0.37739239077190945
Aggregate regression train score with FS: 0.32785662848978125, test score: 0.37739239077190945
(228, 3)
Full aggregate regression train score: 0.32853694589428184, test score: 0.36273367640935494
Aggregate regression train score with FS: 0.32853694589428184, test score: 0.36273367640935494
(228, 4)
Full aggregate regression train score: 0.334498400332087, test score: 0.36992284966717515
Aggregate regression train score with FS: 0.334498400332087, test score: 0.36992284966717515
(228, 5)
Full aggregate regression tr

Full aggregate regression train score: 0.5103173844323213, test score: 0.11006691374655209
Aggregate regression train score with FS: 0.5103173844323213, test score: 0.11006691374655209
(228, 44)
Full aggregate regression train score: 0.5103236104360857, test score: 0.1107438099954059
Aggregate regression train score with FS: 0.5103236104360857, test score: 0.1107438099954059
(228, 45)
Full aggregate regression train score: 0.5172258130977394, test score: 0.12414394820501362
Aggregate regression train score with FS: 0.5172258130977394, test score: 0.12414394820501362
(228, 46)
Full aggregate regression train score: 0.5174634649894193, test score: 0.1160491479080138
Aggregate regression train score with FS: 0.5174634649894193, test score: 0.1160491479080138
(228, 47)
Full aggregate regression train score: 0.5174677324829426, test score: 0.11602561709848547
Aggregate regression train score with FS: 0.5174677324829426, test score: 0.11602561709848547
(228, 48)
Full aggregate regression tra

Full aggregate regression train score: 0.476261718197464, test score: 0.21322206264203758
Aggregate regression train score with FS: 0.476261718197464, test score: 0.21322206264203758
(228, 36)
Full aggregate regression train score: 0.4762674264581884, test score: 0.21299942087578017
Aggregate regression train score with FS: 0.4762674264581884, test score: 0.21299942087578017
(228, 37)
Full aggregate regression train score: 0.47761164483620056, test score: 0.1873158815540622
Aggregate regression train score with FS: 0.47761164483620056, test score: 0.1873158815540622
(228, 38)
Full aggregate regression train score: 0.4827933514235423, test score: 0.19845584103467862
Aggregate regression train score with FS: 0.4827933514235423, test score: 0.19845584103467862
(228, 39)
Full aggregate regression train score: 0.4829961689848491, test score: 0.20276005699961308
Aggregate regression train score with FS: 0.4829961689848491, test score: 0.20276005699961308
(228, 40)
Full aggregate regression t

Full aggregate regression train score: 0.463330183163843, test score: 0.2288510025398156
Aggregate regression train score with FS: 0.463330183163843, test score: 0.2288510025398156
(228, 28)
Full aggregate regression train score: 0.48424989614219693, test score: 0.2241486387270185
Aggregate regression train score with FS: 0.48424989614219693, test score: 0.2241486387270185
(228, 29)
Full aggregate regression train score: 0.5009936178522947, test score: 0.2542855378683494
Aggregate regression train score with FS: 0.5009936178522947, test score: 0.2542855378683494
(228, 30)
Full aggregate regression train score: 0.502433965377574, test score: 0.257109482400974
Aggregate regression train score with FS: 0.502433965377574, test score: 0.257109482400974
(228, 31)
Full aggregate regression train score: 0.5026771852215572, test score: 0.26173816040623954
Aggregate regression train score with FS: 0.5026771852215572, test score: 0.26173816040623954
(228, 32)
Full aggregate regression train score

Full aggregate regression train score: 0.441827196846851, test score: 0.23054110244045167
Aggregate regression train score with FS: 0.441827196846851, test score: 0.23054110244045167
(228, 21)
Full aggregate regression train score: 0.45817971781118316, test score: 0.14848251654379818
Aggregate regression train score with FS: 0.45817971781118316, test score: 0.14848251654379818
(228, 22)
Full aggregate regression train score: 0.4582631859449514, test score: 0.14835961581582213
Aggregate regression train score with FS: 0.4582631859449514, test score: 0.14835961581582213
(228, 23)
Full aggregate regression train score: 0.45836745091393083, test score: 0.14411730051780935
Aggregate regression train score with FS: 0.45836745091393083, test score: 0.14411730051780935
(228, 24)
Full aggregate regression train score: 0.45928002007743884, test score: 0.14286299633386246
Aggregate regression train score with FS: 0.45928002007743884, test score: 0.14286299633386246
(228, 25)
Full aggregate regres

...

Full aggregate regression train score: 0.3012520739774408, test score: 0.36149564501516507
Aggregate regression train score with FS: 0.3012520739774408, test score: 0.36149564501516507
(228, 3)
Full aggregate regression train score: 0.30243443517775837, test score: 0.34285859529760143
Aggregate regression train score with FS: 0.30243443517775837, test score: 0.34285859529760143
(228, 4)
Full aggregate regression train score: 0.30324886564371356, test score: 0.3474801690879349
Aggregate regression train score with FS: 0.30324886564371356, test score: 0.3474801690879349
(228, 5)
Full aggregate regression train score: 0.3038734513843666, test score: 0.35565353934811106
Aggregate regression train score with FS: 0.3038734513843666, test score: 0.35565353934811106
(228, 6)
Full aggregate regression train score: 0.3207728860330026, test score: 0.3470441059330591
Aggregate regression train score with FS: 0.3207728860330026, test score: 0.3470441059330591
(228, 7)
Full aggregate regression trai

Full aggregate regression train score: 0.4453889856183796, test score: 0.1769312414545713
Aggregate regression train score with FS: 0.4453889856183796, test score: 0.1769312414545713
(228, 46)
Full aggregate regression train score: 0.4462868586236195, test score: 0.1784423896337225
Aggregate regression train score with FS: 0.4462868586236195, test score: 0.1784423896337225
(228, 47)
Full aggregate regression train score: 0.4463226371628234, test score: 0.1785664167996317
Aggregate regression train score with FS: 0.4463226371628234, test score: 0.1785664167996317
(228, 48)
Full aggregate regression train score: 0.4636533360024896, test score: 0.19209541471636038
Aggregate regression train score with FS: 0.4636533360024896, test score: 0.19209541471636038
(228, 49)
Full aggregate regression train score: 0.4636591709765925, test score: 0.19177324225745418
Aggregate regression train score with FS: 0.4636591709765925, test score: 0.19177324225745418
(228, 50)
0.36149564501516507
(228, 3)
##

Full aggregate regression train score: 0.4881781108218074, test score: 0.17221254476424241
Aggregate regression train score with FS: 0.4881781108218074, test score: 0.17221254476424241
(228, 39)
Full aggregate regression train score: 0.49304980715947744, test score: 0.17046588520439665
Aggregate regression train score with FS: 0.49304980715947744, test score: 0.17046588520439665
(228, 40)
Full aggregate regression train score: 0.502700945346656, test score: 0.13071315109491477
Aggregate regression train score with FS: 0.502700945346656, test score: 0.13071315109491477
(228, 41)
Full aggregate regression train score: 0.5048623887585676, test score: 0.1203786462285995
Aggregate regression train score with FS: 0.5048623887585676, test score: 0.1203786462285995
(228, 42)
Full aggregate regression train score: 0.506011066527315, test score: 0.11525396174962554
Aggregate regression train score with FS: 0.506011066527315, test score: 0.11525396174962554
(228, 43)
Full aggregate regression tra

Full aggregate regression train score: 0.4672318474061544, test score: 0.16356957507264647
Aggregate regression train score with FS: 0.4672318474061544, test score: 0.16356957507264647
(228, 31)
Full aggregate regression train score: 0.46760430786355955, test score: 0.16444595117052263
Aggregate regression train score with FS: 0.46760430786355955, test score: 0.16444595117052263
(228, 32)
Full aggregate regression train score: 0.47021190821403147, test score: 0.17989950791563591
Aggregate regression train score with FS: 0.47021190821403147, test score: 0.17989950791563591
(228, 33)
Full aggregate regression train score: 0.47747230399715557, test score: 0.11073409859186067
Aggregate regression train score with FS: 0.47747230399715557, test score: 0.11073409859186067
(228, 34)
Full aggregate regression train score: 0.47750865222813454, test score: 0.10803529366354836
Aggregate regression train score with FS: 0.47750865222813454, test score: 0.10803529366354836
(228, 35)
Full aggregate re

Full aggregate regression train score: 0.3858512192980934, test score: 0.295950619379457
Aggregate regression train score with FS: 0.3858512192980934, test score: 0.295950619379457
(228, 23)
Full aggregate regression train score: 0.3961310298778262, test score: 0.23217837907110994
Aggregate regression train score with FS: 0.3961310298778262, test score: 0.23217837907110994
(228, 24)
Full aggregate regression train score: 0.39707940348560666, test score: 0.23759516467894282
Aggregate regression train score with FS: 0.39707940348560666, test score: 0.23759516467894282
(228, 25)
Full aggregate regression train score: 0.40416059759082534, test score: 0.28154792347295776
Aggregate regression train score with FS: 0.40416059759082534, test score: 0.28154792347295776
(228, 26)
Full aggregate regression train score: 0.40645910637392746, test score: 0.2538493464460585
Aggregate regression train score with FS: 0.40645910637392746, test score: 0.2538493464460585
(228, 27)
Full aggregate regression

In [35]:
linear_r2 = [0.42566301160582065,0.35821575019621354,0.37210020625606555,0.39536152483534615,0.37027968745993034]
poly_r2 = [0.4227023510810889,0.36023372191771696,0.4025986907366008,0.39064261018084945,0.36821229997640637]
sigmoid_r2 = [0.4209891256644783,0.36149564501516485,0.3737606521182656,0.38423040090739946,0.3765177020663276]

linear_n = [11,11,6,9,12]
poly_n = [34,32,9,9,25]
sigmoid_n = [11,3,3,3,10]

compute_CI(linear_r2,5)
compute_CI(poly_r2,5)
compute_CI(sigmoid_r2,5)

compute_CI(linear_n,5)
compute_CI(poly_n,5)
compute_CI(sigmoid_n,5)


0.3843240360706752 +- 0.020960256462471853
0.3888779347785325 +- 0.01992279503719367
0.38339870515432717 +- 0.01767792115256536
9.8 +- 1.8717743453739286
21.8 +- 9.528100083437412
6.0 +- 3.2325148104842456


In [41]:
from sklearn.manifold import LocallyLinearEmbedding as LLE
## r2 = []
for curr_seed in [0,1,2,3,4]: 
    best_r2 = 0
    best_num = 0
    for i in range(50):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = LLE(n_components=i+1,n_neighbors=10)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        actual_r2 = compare_methods(trainVal_reduced, test_reduced, curr_df_trainVal_withTar, df_test_withTar, list(trainVal_reduced.columns))
        r2.append([curr_seed,actual_r2])
        print(test_reduced.shape)
        if actual_r2> best_r2:
            best_r2=actual_r2
            best_num = test_reduced.shape
    print(best_r2)
    print(best_num)

Full aggregate regression train score: 0.14241891552877273, test score: 0.170290034590604
Aggregate regression train score with FS: 0.14241891552877273, test score: 0.170290034590604
(228, 1)
Full aggregate regression train score: 0.18821607344367275, test score: 0.19228898620428514
Aggregate regression train score with FS: 0.18821607344367275, test score: 0.19228898620428514
(228, 2)
Full aggregate regression train score: 0.21953476076522915, test score: 0.2528704150405403
Aggregate regression train score with FS: 0.21953476076522915, test score: 0.2528704150405403
(228, 3)
Full aggregate regression train score: 0.22000091851693881, test score: 0.24944180514405967
Aggregate regression train score with FS: 0.22000091851693881, test score: 0.24944180514405967
(228, 4)
Full aggregate regression train score: 0.24282123793079136, test score: 0.26580468200139873
Aggregate regression train score with FS: 0.24282123793079136, test score: 0.26580468200139873
(228, 5)
Full aggregate regression 

Full aggregate regression train score: 0.4752847815105019, test score: 0.337844856028239
Aggregate regression train score with FS: 0.4752847815105019, test score: 0.337844856028239
(228, 45)
Full aggregate regression train score: 0.4767242062614535, test score: 0.3433472757694348
Aggregate regression train score with FS: 0.4767242062614535, test score: 0.3433472757694348
(228, 46)
Full aggregate regression train score: 0.4939340139321804, test score: 0.33660828167672685
Aggregate regression train score with FS: 0.4939340139321804, test score: 0.33660828167672685
(228, 47)
Full aggregate regression train score: 0.4982478480081469, test score: 0.34778844530028974
Aggregate regression train score with FS: 0.4982478480081469, test score: 0.34778844530028974
(228, 48)
Full aggregate regression train score: 0.49832338743385873, test score: 0.3470762240185563
Aggregate regression train score with FS: 0.49832338743385873, test score: 0.3470762240185563
(228, 49)
Full aggregate regression train

Full aggregate regression train score: 0.41970813505619686, test score: 0.2076462433422802
Aggregate regression train score with FS: 0.41970813505619686, test score: 0.2076462433422802
(228, 37)
Full aggregate regression train score: 0.42192582676744583, test score: 0.21938121594531934
Aggregate regression train score with FS: 0.42192582676744583, test score: 0.21938121594531934
(228, 38)
Full aggregate regression train score: 0.4219422040983186, test score: 0.2206849239067129
Aggregate regression train score with FS: 0.4219422040983186, test score: 0.2206849239067129
(228, 39)
Full aggregate regression train score: 0.4275417479698216, test score: 0.22685423127265447
Aggregate regression train score with FS: 0.4275417479698216, test score: 0.22685423127265447
(228, 40)
Full aggregate regression train score: 0.4306388459882057, test score: 0.2404440382302755
Aggregate regression train score with FS: 0.4306388459882057, test score: 0.2404440382302755
(228, 41)
Full aggregate regression t

Full aggregate regression train score: 0.5211128964136267, test score: 0.3465309533806651
Aggregate regression train score with FS: 0.5211128964136267, test score: 0.3465309533806651
(228, 29)
Full aggregate regression train score: 0.5233248091036775, test score: 0.34960841600702663
Aggregate regression train score with FS: 0.5233248091036775, test score: 0.34960841600702663
(228, 30)
Full aggregate regression train score: 0.5383911343404253, test score: 0.34909973817691065
Aggregate regression train score with FS: 0.5383911343404253, test score: 0.34909973817691065
(228, 31)
Full aggregate regression train score: 0.5474937013362157, test score: 0.33142209714960713
Aggregate regression train score with FS: 0.5474937013362157, test score: 0.33142209714960713
(228, 32)
Full aggregate regression train score: 0.5481985608653821, test score: 0.3320189937631358
Aggregate regression train score with FS: 0.5481985608653821, test score: 0.3320189937631358
(228, 33)
Full aggregate regression tra

Full aggregate regression train score: 0.42639127092991924, test score: 0.28583946817492545
Aggregate regression train score with FS: 0.42639127092991924, test score: 0.28583946817492545
(228, 23)
Full aggregate regression train score: 0.43485516497141485, test score: 0.2771655249722773
Aggregate regression train score with FS: 0.43485516497141485, test score: 0.2771655249722773
(228, 24)
Full aggregate regression train score: 0.4464014793594444, test score: 0.29851319526046816
Aggregate regression train score with FS: 0.4464014793594444, test score: 0.29851319526046816
(228, 25)
Full aggregate regression train score: 0.44687226710180483, test score: 0.3007484824061051
Aggregate regression train score with FS: 0.44687226710180483, test score: 0.3007484824061051
(228, 26)
Full aggregate regression train score: 0.4695770381915674, test score: 0.27917119359875286
Aggregate regression train score with FS: 0.4695770381915674, test score: 0.27917119359875286
(228, 27)
Full aggregate regressi

Full aggregate regression train score: 0.2522962611024614, test score: 0.23372418511616488
Aggregate regression train score with FS: 0.2522962611024614, test score: 0.23372418511616488
(228, 15)
Full aggregate regression train score: 0.2570398559464716, test score: 0.2563926346847313
Aggregate regression train score with FS: 0.2570398559464716, test score: 0.2563926346847313
(228, 16)
Full aggregate regression train score: 0.2583797655263722, test score: 0.25879443881575737
Aggregate regression train score with FS: 0.2583797655263722, test score: 0.25879443881575737
(228, 17)
Full aggregate regression train score: 0.25853817893781195, test score: 0.2584936431113354
Aggregate regression train score with FS: 0.25853817893781195, test score: 0.2584936431113354
(228, 18)
Full aggregate regression train score: 0.2879904848020973, test score: 0.2119401109704604
Aggregate regression train score with FS: 0.2879904848020973, test score: 0.2119401109704604
(228, 19)
Full aggregate regression tra

In [43]:
best_lle_r2 = [0.34804058383161696,0.2584045442613818,0.34960841600702663,0.3349244497444306,0.26021386822034054]
best_lle_n = [50,18,30,48,29]

compute_CI(best_lle_r2,5)
compute_CI(best_lle_n,5)


0.31023837241295926 +- 0.036726041921633694
35.0 +- 10.692334450436912


# Classification

In [46]:
### Load and standardize target
target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path='../PaperNon_Gen_LinCFA/droughts/Emiliani1.csv')
target_df_train['mean_std'] = target_df_train.apply(lambda x:np.sign(x.mean_std),axis=1)
target_df_val['mean_std'] = target_df_val.apply(lambda x:np.sign(x.mean_std),axis=1)
target_df_test['mean_std'] = target_df_test.apply(lambda x:np.sign(x.mean_std),axis=1)
target_df_trainVal['mean_std'] = target_df_trainVal.apply(lambda x:np.sign(x.mean_std),axis=1)


target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.379890    0.50  2001     1 -0.382765
1    2001-01-13  0.482679    0.58  2001     2  0.319215
2    2001-01-21  0.516259    0.59  2001     3  0.548542
3    2001-01-29  0.434421    0.50  2001     5 -0.010351
4    2001-02-06  0.494805    0.54  2001     6  0.402030
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.427085    0.43  2009    48 -0.060454
407  2009-12-05  0.547380    0.57  2009    49  0.761079
408  2009-12-13  0.531070    0.58  2009    50  0.649694
409  2009-12-21  0.295704    0.00  2009    52 -0.957702
410  2009-12-29  0.027861    0.00  2009    53 -2.786888

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))


In [48]:
### Load and standardize features
variables_list = ['cyclostationary_mean_tg', 
                 'cyclostationary_mean_tg_1w',
                 'cyclostationary_mean_tg_4w', 
                 'cyclostationary_mean_tg_8w',
                 'cyclostationary_mean_tg_12w', 
                 'cyclostationary_mean_tg_16w',
                 'cyclostationary_mean_tg_24w',
                 'cyclostationary_mean_rr', 
                 'cyclostationary_mean_rr_1w',
                 'cyclostationary_mean_rr_4w', 
                 'cyclostationary_mean_rr_8w',
                 'cyclostationary_mean_rr_12w', 
                 'cyclostationary_mean_rr_16w',
                 'cyclostationary_mean_rr_24w'
                 ]

df_train = pd.DataFrame()
df_val = pd.DataFrame()
df_test = pd.DataFrame()
df_trainVal = pd.DataFrame()

for variable in variables_list:
    df_train_unfolded_std, df_val_unfolded_std, df_test_unfolded_std,df_trainVal_unfolded_std = prepare_features('../PaperNon_Gen_LinCFA/droughts/Emiliani1_aggreg.csv',variable,False,max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')
    df_train_unfolded_std = df_train_unfolded_std.add_prefix(variable)
    df_val_unfolded_std = df_val_unfolded_std.add_prefix(variable)
    df_test_unfolded_std = df_test_unfolded_std.add_prefix(variable)
    df_trainVal_unfolded_std = df_trainVal_unfolded_std.add_prefix(variable)
    df_train = pd.concat((df_train,df_train_unfolded_std),axis=1)
    df_val = pd.concat((df_val,df_val_unfolded_std),axis=1)
    df_test = pd.concat((df_test,df_test_unfolded_std),axis=1)
    df_trainVal = pd.concat((df_trainVal,df_trainVal_unfolded_std),axis=1)
    
df_trainVal

,cyclostationary_mean_tgmean_12.149860342381333_43.74986055078544,cyclostationary_mean_tgmean_12.149860342381333_43.8498605504681,cyclostationary_mean_tgmean_12.149860342381333_43.94986055015075,cyclostationary_mean_tgmean_12.149860342381333_44.04986054983341,cyclostationary_mean_tgmean_12.149860342381333_44.14986054951606,cyclostationary_mean_tgmean_12.149860342381333_44.24986054919872,cyclostationary_mean_tgmean_12.149860342381333_44.34986054888138,cyclostationary_mean_tgmean_12.149860342381333_44.44986054856403,cyclostationary_mean_tgmean_12.149860342381333_44.54986054824669,cyclostationary_mean_tgmean_12.149860342381333_44.64986054792934,...,cyclostationary_mean_rr_24wmean_11.349860345581744_44.849860547294654,cyclostationary_mean_rr_24wmean_11.449860345181692_44.14986054951606,cyclostationary_mean_rr_24wmean_11.449860345181692_44.24986054919872,cyclostationary_mean_rr_24wmean_11.449860345181692_44.34986054888138,cyclostationary_mean_rr_24wmean_11.449860345181692_44.44986054856403,cyclostationary_mean_rr_24wmean_11.449860345181692_44.54986054824669,cyclostationary_mean_rr_24wmean_11.449860345181692_44.64986054792934,cyclostationary_mean_rr_24wmean_11.449860345181692_44.749860547612,cyclostationary_mean_rr_24wmean_11.449860345181692_44.849860547294654,cyclostationary_mean_rr_24wmean_10.749860347982052_44.24986054919872
0,0.719982,0.737633,0.791074,0.848973,0.983768,1.172439,1.201792,1.127842,0.986823,0.794499,...,2.331970,6.168345,6.215139,5.406085,4.240808,4.540304,2.472063,2.105030,2.296531,3.363161
1,1.845931,2.162888,2.278888,2.313132,2.330406,2.363919,2.119857,2.008556,1.797622,1.626867,...,1.670322,3.527914,3.841691,3.108603,2.947959,2.824932,1.593425,1.462249,1.728906,3.004583
2,-0.312107,-0.404970,-0.361520,-0.116017,0.037568,0.199148,0.215971,0.233859,0.106281,-0.117314,...,1.118337,1.915294,2.358315,2.981883,2.858264,2.382361,1.067353,1.006560,1.055914,1.950734
3,1.666516,1.661740,1.601745,1.595421,1.579804,1.588420,1.391649,1.185715,1.141724,0.968893,...,0.534584,2.429799,2.789340,2.202089,1.991279,1.546552,0.466434,0.489244,0.506180,2.157730
4,-0.031846,-0.003213,-0.013352,0.122002,0.295776,0.473944,0.532393,0.421560,0.238369,-0.025356,...,1.056396,2.718182,3.000691,3.082463,3.061446,2.457115,1.281718,1.138486,1.142762,2.350811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,0.578850,0.609135,0.689113,0.967141,1.273710,1.237462,1.007356,1.032989,1.194737,1.011196,...,0.690343,1.041006,0.388026,0.123181,0.347596,0.465708,-0.240666,0.449174,0.650788,1.164679
635,1.243632,1.043466,0.783567,0.922940,1.124030,1.209410,1.205085,1.367888,1.900526,1.924692,...,0.666434,0.955184,0.294397,0.090659,0.410255,0.581733,-0.169828,0.462829,0.652467,1.096017
636,0.267909,0.598843,0.778573,1.011130,1.340522,1.491446,1.432183,1.526572,1.965525,1.990127,...,0.894777,1.047647,0.425352,0.262704,0.623916,0.826442,0.059008,0.692457,0.885935,1.242053
637,0.957833,1.053095,1.249241,1.296098,1.319729,1.135229,0.976383,1.250418,1.632408,1.738173,...,0.878993,1.036936,0.456154,0.304824,0.500595,0.733759,0.026283,0.689393,0.807281,1.119456


In [49]:
### Together
df_trainVal_withTar = pd.concat((df_trainVal,target_df_trainVal['mean_std']), axis=1)
df_test_withTar = pd.concat((df_test,target_df_test.reset_index()['mean_std']), axis=1)

In [50]:
from sklearn.linear_model import LogisticRegression

# PCA

In [51]:
# explain 95% of variance
from sklearn.decomposition import PCA
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression

r2 = []
for curr_seed in [0,1,2,3,4]: 
    curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
    curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
    pca = PCA(n_components=0.95)
    trainVal_pca = pd.DataFrame(pca.fit_transform(curr_df_trainVal))
    test_pca = pd.DataFrame(pca.transform(df_test))
    print(test_pca.shape)
    mod = LogisticRegression(random_state=0).fit(trainVal_pca, curr_df_trainVal_withTar.mean_std)
    print(mod.score(test_pca, df_test_withTar.mean_std))
    

(228, 29)
0.6798245614035088
(228, 30)
0.6973684210526315
(228, 30)
0.6929824561403509
(228, 29)
0.6578947368421053
(228, 29)
0.706140350877193


In [52]:
n_pca95 = [29,30,30,29,29]
r2_pca95 = [0.6798245614035088,0.6973684210526315,0.6929824561403509,0.6578947368421053,0.706140350877193]
compute_CI(n_pca95,5)
compute_CI(r2_pca95,5)

29.4 +- 0.42941448508405017
0.6868421052631579 +- 0.014709799749812812


# LDA

In [53]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [54]:
from sklearn.manifold import LocallyLinearEmbedding as LLE
## r2 = []
for curr_seed in [0,1,2,3,4]: 
    for i in range(1):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = LDA(n_components=i+1)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal,curr_df_trainVal_withTar.mean_std))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        print(test_reduced.shape)
        mod = LogisticRegression(random_state=0).fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
        print(mod.score(test_reduced, df_test_withTar.mean_std))
        

(228, 1)
0.5877192982456141
(228, 1)
0.5350877192982456
(228, 1)
0.5745614035087719
(228, 1)
0.5482456140350878
(228, 1)
0.5175438596491229


In [55]:
compute_CI([0.5877192982456141,0.5350877192982456,0.5745614035087719,0.5482456140350878,0.5175438596491229],5)

0.5526315789473684 +- 0.022416906516135423


# Kernel PCA

In [56]:
for curr_kernel in ['linear', 'poly', 'sigmoid']:
    for curr_seed in [0,1,2,3,4]: 
        best_score = 0
        best_num = 0
        print(f'#########################{curr_kernel}, {curr_seed}#############################')
        for i in range(50):
            curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
            curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
            dimRedMethod = KernelPCA(n_components=i+1, kernel=curr_kernel)
            trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal))
            test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
            print(test_reduced.shape)
            mod = LogisticRegression(random_state=0).fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
            actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
            print(actual_score)
            if actual_score> best_score:
                best_score=actual_score
                best_num = test_reduced.shape
        print(best_score)
        print(best_num)


#########################linear, 0#############################
(228, 1)
0.6491228070175439
(228, 2)
0.6798245614035088
(228, 3)
0.7412280701754386
(228, 4)
0.7412280701754386
(228, 5)
0.7412280701754386
(228, 6)
0.75
(228, 7)
0.7719298245614035
(228, 8)
0.7631578947368421
(228, 9)
0.7587719298245614
(228, 10)
0.75
(228, 11)
0.7543859649122807
(228, 12)
0.75
(228, 13)
0.7149122807017544
(228, 14)
0.7236842105263158
(228, 15)
0.706140350877193
(228, 16)
0.7105263157894737
(228, 17)
0.7192982456140351
(228, 18)
0.7149122807017544
(228, 19)
0.6842105263157895
(228, 20)
0.7149122807017544
(228, 21)
0.7236842105263158
(228, 22)
0.7236842105263158
(228, 23)
0.7192982456140351
(228, 24)
0.6973684210526315
(228, 25)
0.7017543859649122
(228, 26)
0.7149122807017544
(228, 27)
0.7017543859649122
(228, 28)
0.6798245614035088
(228, 29)
0.6798245614035088
(228, 30)
0.6798245614035088
(228, 31)
0.6754385964912281
(228, 32)
0.6666666666666666
(228, 33)
0.6754385964912281
(228, 34)
0.6798245614035088
(2

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

(228, 49)
0.6798245614035088
(228, 50)
0.6710526315789473
0.7719298245614035
(228, 7)
#########################linear, 1#############################
(228, 1)


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6578947368421053
(228, 2)
0.6666666666666666
(228, 3)
0.7280701754385965
(228, 4)
0.7192982456140351
(228, 5)
0.7412280701754386
(228, 6)
0.7587719298245614
(228, 7)
0.7587719298245614
(228, 8)
0.7412280701754386
(228, 9)
0.7412280701754386
(228, 10)
0.7412280701754386
(228, 11)
0.7456140350877193
(228, 12)
0.7368421052631579
(228, 13)
0.7368421052631579
(228, 14)
0.7368421052631579
(228, 15)
0.7324561403508771
(228, 16)
0.7280701754385965
(228, 17)
0.7412280701754386
(228, 18)
0.7324561403508771
(228, 19)
0.7280701754385965
(228, 20)
0.7236842105263158
(228, 21)
0.7149122807017544
(228, 22)
0.7192982456140351
(228, 23)
0.6973684210526315
(228, 24)
0.7017543859649122
(228, 25)
0.706140350877193
(228, 26)
0.6842105263157895
(228, 27)
0.6798245614035088
(228, 28)
0.6885964912280702
(228, 29)
0.6973684210526315
(228, 30)
0.6973684210526315
(228, 31)
0.6622807017543859
(228, 32)
0.706140350877193
(228, 33)
0.706140350877193
(228, 34)
0.6842105263157895
(228, 35)
0.6885964912280702
(228, 

...

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

(228, 49)
0.618421052631579
(228, 50)
0.618421052631579
0.7631578947368421
(228, 8)
#########################linear, 4#############################
(228, 1)
0.6535087719298246
(228, 2)
0.6666666666666666


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(228, 3)
0.7192982456140351
(228, 4)
0.7149122807017544
(228, 5)
0.7192982456140351
(228, 6)
0.7587719298245614
(228, 7)
0.7412280701754386
(228, 8)
0.7324561403508771
(228, 9)
0.7280701754385965
(228, 10)
0.7236842105263158
(228, 11)
0.7368421052631579
(228, 12)
0.7412280701754386
(228, 13)
0.7280701754385965
(228, 14)
0.7456140350877193
(228, 15)
0.7236842105263158
(228, 16)
0.7324561403508771
(228, 17)
0.706140350877193
(228, 18)
0.7105263157894737
(228, 19)
0.706140350877193
(228, 20)
0.7105263157894737
(228, 21)
0.7017543859649122
(228, 22)
0.7105263157894737
(228, 23)
0.7236842105263158
(228, 24)
0.7236842105263158
(228, 25)
0.706140350877193
(228, 26)
0.706140350877193
(228, 27)
0.706140350877193
(228, 28)
0.7280701754385965
(228, 29)
0.706140350877193
(228, 30)
0.7236842105263158
(228, 31)
0.7017543859649122
(228, 32)
0.706140350877193
(228, 33)
0.706140350877193
(228, 34)
0.6973684210526315
(228, 35)
0.6798245614035088
(228, 36)
0.6973684210526315
(228, 37)
0.6973684210526315


(228, 25)
0.7543859649122807
(228, 26)
0.7543859649122807
(228, 27)
0.7631578947368421
(228, 28)
0.7587719298245614
(228, 29)
0.7587719298245614
(228, 30)
0.7456140350877193
(228, 31)
0.75
(228, 32)
0.75
(228, 33)
0.75
(228, 34)
0.7543859649122807
(228, 35)
0.75
(228, 36)
0.7192982456140351
(228, 37)
0.7149122807017544
(228, 38)
0.7280701754385965
(228, 39)
0.7324561403508771
(228, 40)
0.7192982456140351
(228, 41)
0.7236842105263158
(228, 42)
0.7236842105263158
(228, 43)
0.6973684210526315
(228, 44)
0.706140350877193
(228, 45)
0.706140350877193
(228, 46)
0.6973684210526315
(228, 47)
0.6929824561403509
(228, 48)
0.7105263157894737
(228, 49)
0.6885964912280702
(228, 50)
0.7105263157894737
0.7631578947368421
(228, 27)
#########################sigmoid, 0#############################
(228, 1)
0.6666666666666666
(228, 2)
0.6842105263157895
(228, 3)
0.7456140350877193
(228, 4)
0.7412280701754386
(228, 5)
0.7543859649122807
(228, 6)
0.7543859649122807
(228, 7)
0.7543859649122807
(228, 8)
0.758

In [58]:
linear_r2 = [0.7719298245614035,0.7587719298245614,0.7543859649122807,0.7631578947368421,0.7587719298245614]
poly_r2 = [0.7675438596491229,0.7456140350877193,0.7587719298245614,0.7456140350877193,0.7631578947368421]
sigmoid_r2 = [0.7763157894736842,0.7675438596491229,0.7587719298245614,0.7587719298245614,0.7587719298245614]

linear_n = [7,6,6,8,6]
poly_n = [23,17,7,26,27]
sigmoid_n = [16,11,11,8,10]

compute_CI(linear_r2,5)
compute_CI(poly_r2,5)
compute_CI(sigmoid_r2,5)

compute_CI(linear_n,5)
compute_CI(poly_n,5)
compute_CI(sigmoid_n,5)


0.7614035087719297 +- 0.005214889779067013
0.7561403508771929 +- 0.0079162436102154
0.7640350877192983 +- 0.006151148401262564
6.6 +- 0.7012309177439341
20.0 +- 6.465029620968491
11.2 +- 2.3124677727484118


# Isomap

In [59]:
from sklearn.manifold import Isomap

for curr_seed in [0,1,2,3,4]: 
    best_score = 0
    best_num = 0
    for i in range(50):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = Isomap(n_components=i+1,n_neighbors=10)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal.values,curr_df_trainVal_withTar.mean_std.values.reshape(-1, 1)))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        print(test_reduced.shape)
        mod = LogisticRegression(random_state=0).fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
        actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
        print(actual_score)
        if actual_score> best_score:
            best_score=actual_score
            best_num = test_reduced.shape
    print(best_score)
    print(best_num)
    

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


(228, 1)
0.6140350877192983


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


(228, 2)
0.6622807017543859
(228, 3)
0.6798245614035088
(228, 4)


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


...

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


(228, 48)
0.6666666666666666


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


(228, 49)
0.6798245614035088
(228, 50)
0.6798245614035088
0.7236842105263158
(228, 16)


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


In [60]:
best_isomap_r2 = [0.7105263157894737,0.7412280701754386,0.7543859649122807,0.75,0.7236842105263158]

best_isomap_n = [36,16,10,5,16]

compute_CI(best_isomap_r2,5)
compute_CI(best_isomap_n,5)


0.7359649122807018 +- 0.014466645816563775
16.6 +- 9.236571485134514


# LLE

In [71]:
best_scores = []
best_nums = []
for curr_seed in [0,1,2,3,4]: 
    best_score = 0
    best_num = 0
    for i in range(50):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = LLE(n_components=i+1,n_neighbors=10)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal.values,curr_df_trainVal_withTar.mean_std.values.reshape(-1, 1)))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        print(test_reduced.shape)
        mod = LogisticRegression(random_state=0).fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
        actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
        print(actual_score)
        if actual_score> best_score:
            best_score=actual_score
            best_num = test_reduced.shape
    print(best_score)
    print(best_num)
    best_scores.append(best_score)
    best_nums.append(best_num[1])
    

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


(228, 1)
0.6622807017543859
(228, 2)
0.6929824561403509


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


(228, 3)
0.6929824561403509
(228, 4)
0.7017543859649122


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


...

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


(228, 46)
0.6842105263157895
(228, 47)
0.6929824561403509


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


(228, 48)
0.6885964912280702
(228, 49)
0.7105263157894737
(228, 50)
0.706140350877193
0.7587719298245614
(228, 21)


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LocallyLinearEmbedding was fitted without feature names
  warnings.warn(


In [75]:
compute_CI(best_scores,5)
compute_CI(best_nums,5)


0.7456140350877194 +- 0.013967644976391662
25.8 +- 13.13521751628042


# NCA

In [76]:
from sklearn.neighbors import NeighborhoodComponentsAnalysis as NCA

best_scores = []
best_nums = []
for curr_seed in [0,1,2,3,4]: 
    best_score = 0
    best_num = 0
    for i in range(50):
        curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
        curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
        dimRedMethod = NCA(n_components=i+1)
        trainVal_reduced = pd.DataFrame(dimRedMethod.fit_transform(curr_df_trainVal.values,curr_df_trainVal_withTar.mean_std.values.reshape(-1, 1)))
        test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
        print(test_reduced.shape)
        mod = LogisticRegression(random_state=0).fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
        actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
        print(actual_score)
        if actual_score> best_score:
            best_score=actual_score
            best_num = test_reduced.shape
    print(best_score)
    print(best_num)
    best_scores.append(best_score)
    best_nums.append(best_num[1])
    

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(228, 1)
0.6140350877192983


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(228, 2)
0.6666666666666666


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(

...

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(228, 48)
0.6798245614035088


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(
/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(228, 49)
0.6754385964912281
(228, 50)
0.6666666666666666
0.7675438596491229
(228, 6)


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NeighborhoodComponentsAnalysis was fitted without feature names
  warnings.warn(


In [77]:
compute_CI(best_scores,5)
compute_CI(best_nums,5)


0.7640350877192983 +- 0.005100262819195603
7.0 +- 0.5543717164502532


# Supervised PCA

In [80]:
for kernel in ['linear', 'poly', 'sigmoid']:
    best_scores = []
    best_nums = []
    for curr_seed in [0,1,2,3,4]: 
        print(f'#########################{kernel}, {curr_seed}#############################')
        best_score = 0
        best_num = 0
        for i in range(50):
            curr_df_trainVal_withTar = resample(df_trainVal_withTar, random_state=curr_seed)
            curr_df_trainVal = curr_df_trainVal_withTar.iloc[:,:-1]
            dimRedMethod = spca(num_components=i+1, kernel='linear', degree=3, gamma=None, coef0=1)
            trainVal_reduced = pd.DataFrame(dimRedMethod.fit_and_transform(curr_df_trainVal.values,curr_df_trainVal_withTar.mean_std.values.reshape(-1, 1)))
            test_reduced = pd.DataFrame(dimRedMethod.transform(df_test))
            print(test_reduced.shape)
            mod = LogisticRegression(random_state=0).fit(trainVal_reduced, curr_df_trainVal_withTar.mean_std)
            actual_score = mod.score(test_reduced, df_test_withTar.mean_std)
            print(actual_score)
            if actual_score> best_score:
                best_score=actual_score
                best_num = test_reduced.shape
        print(best_score)
        print(best_num)
        best_scores.append(best_score)
        best_nums.append(best_num[1])
    compute_CI(best_scores,5)
    compute_CI(best_nums,5)

#########################linear, 0#############################
(228, 1)
0.6885964912280702
(228, 2)
0.6973684210526315
(228, 3)
0.706140350877193
(228, 4)
0.7105263157894737
(228, 5)
0.6885964912280702
(228, 6)
0.7280701754385965
(228, 7)
0.7017543859649122
(228, 8)
0.6798245614035088
(228, 9)
0.6710526315789473
(228, 10)
0.7017543859649122
(228, 11)
0.7017543859649122
(228, 12)
0.7236842105263158
(228, 13)
0.6140350877192983
(228, 14)
0.7236842105263158


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(228, 15)
0.7192982456140351
(228, 16)
0.7149122807017544
(228, 17)
0.7017543859649122
(228, 18)
0.7236842105263158
(228, 19)
0.6885964912280702
(228, 20)
0.6842105263157895


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(228, 21)
0.6798245614035088


...

/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(228, 49)
0.6973684210526315


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(228, 50)
0.7236842105263158
0.7543859649122807
(228, 11)
0.7473684210526316 +- 0.00929057611535734
14.0 +- 6.369246109234592


/Users/paolo/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
